In [1]:
# Python
import pandas as pd
import numpy as np
from fbprophet import Prophet
%matplotlib inline

In [2]:
import databricks.koalas as ks

df = ks.read_table("store_data.product_sales_city")
df.head()

In [3]:
# Python
def forecast(tdf):
  forecast_df = tdf[['ds', 'y']]
  
  m = Prophet()
  m.fit(forecast_df)
  
  future = m.make_future_dataframe(periods=365)
  forecast = m.predict(future)
  
  # add city ID and forecast date to forecast
  from datetime import date
  forecast['store_id'] = tdf['store_id'].iloc[0]
  forecast['forecast_date'] = date.today()

  # store forecast in delta table
  spark.createDataFrame(forecast).write.format("delta").insertInto("store_data.forecast_city")
  
  m.plot(forecast)
  
  return m, forecast

In [4]:
# apply forecast to each store
df.groupby("store_id").apply(forecast)